In [ ]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=1)
scipy.__version__

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [ ]:
%%time
print("\n\n\n\n\n\n optimize_acqf \n\n")

acqf = qNoisyExpectedImprovement(model, train_X, cache_root=False)

# eval and maximise acq functions
with cProfile.Profile() as pr:
    optimize_acqf(
        acq_function=acqf,
        bounds=bounds_norm,
        q=5,
        num_restarts=5,
        raw_samples=5,
        sequential=False,
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__

In [ ]:
%%time
print("\n\n\n\n\n\n calling minimize directly \n\n")


def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((5, 5, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

bounds = Bounds(
    lb=np.zeros(50),
    ub=np.ones(50),
    keep_feasible=True,
)
x0 = np.random.random(50)

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__

In [ ]:
%%time
print("\n\n\n\n\n\n 100 acqf evaluations \n\n")

with cProfile.Profile() as pr:
    for _ in range(1):
        test_X = torch.rand(20, 5, 2, requires_grad=True)
        acqf(test_X).sum().backward()

# sortby = SortKey.CUMULATIVE
# ps = pstats.Stats(pr).sort_stats(sortby)
# ps.print_stats(.1)

scipy.__version__